In [0]:
## This Cell is related to date preprocessing and is completely copied from original kipf's github code!

import numpy as np
import scipy.sparse as sp
import torch,torch.nn,torch.sparse,torch.nn.functional,torch.distributions
from sklearn.metrics import roc_auc_score,average_precision_score
from input_data import load_data
from preprocessing import preprocess_graph, sparse_to_tuple, mask_test_edges
np.set_printoptions(threshold=np.inf)



adj, features = load_data("citeseer")


# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train


# Some preprocessing
adj_norm = preprocess_graph(adj)


num_nodes = adj.shape[0]

features = sparse_to_tuple(features.tocoo())
num_features = features[2][1]
features_nonzero = features[1].shape[0]



pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)



adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)




In [0]:
#Model definition Cell

test_edges_false = np.array(test_edges_false)
val_edges_false = np.array(val_edges_false)
adj_label_tensor = torch.sparse.FloatTensor(torch.LongTensor(adj_label[0].transpose()),torch.FloatTensor(adj_label[1]),torch.Size(adj_label[2])).to_dense()
adj_norm_tensor  = torch.sparse.FloatTensor(torch.LongTensor(adj_norm[0].transpose()) ,torch.FloatTensor(adj_norm[1]), torch.Size(adj_norm[2])).to_dense()
features_tensor  = torch.sparse.FloatTensor(torch.LongTensor(features[0].transpose()) ,torch.FloatTensor(features[1]), torch.Size(features[2])).to_dense()

# from the previous cell following arrays are obtained : 
#adj_label
#adj_norm
#features
#val_edges_false
#test_edges
#test_edges_false




class Model(torch.nn.Module):

    def __init__(self,first_layer_dim=30,embedding_dim=15,A_tilda=None ,**kwargs):
        super(Model, self).__init__(**kwargs)


        self.W0 = torch.nn.Linear(num_features, first_layer_dim)
        self.W1_mean = torch.nn.Linear(first_layer_dim, embedding_dim)
        self.W1_log_std  = torch.nn.Linear(first_layer_dim, embedding_dim)

        self.A_tilda = A_tilda

        self.normal_dist = torch.distributions.MultivariateNormal(loc=torch.zeros(embedding_dim),scale_tril=torch.eye(embedding_dim))

        self.optimizer = torch.optim.Adam(params=list(self.W0.parameters())+list(self.W1_mean.parameters())+list(self.W1_log_std.parameters()),lr=0.01)
        self.recon = None


    def train(self,x,A):


        first_layer = torch.nn.functional.relu(torch.matmul(self.A_tilda,self.W0(x)))
        z_mean      = torch.matmul(self.A_tilda,self.W1_mean(first_layer))
        z_log_std   = torch.matmul(self.A_tilda,self.W1_log_std(first_layer))


        z = z_mean +self.normal_dist.sample((x.shape[0],)) * torch.exp(z_log_std)


        recon = torch.matmul(z,z.transpose(dim0=0,dim1=1))
        self.recon = recon

        recon_loss = (  A    *    torch.nn.functional.logsigmoid(recon)  ).mean()*pos_weight + ( (1-A) * torch.nn.functional.logsigmoid(-recon)).mean()
        recon_loss = -norm*recon_loss

        kl_loss = (0.5 / x.shape[0]) * torch.mean(torch.sum(1 + 2 * z_log_std - z_mean**2 -torch.exp(z_log_std)**2, 1))
        kl_loss = -kl_loss

        loss = recon_loss+kl_loss


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()


    def evaluate(self,pos_edge,neg_edge):
        pred_pos_edge = self.recon[pos_edge[:, 0].reshape(-1), pos_edge[:, 1].reshape(-1)].sigmoid().detach().numpy()
        pred_neg_edge = self.recon[neg_edge[:, 0].reshape(-1), neg_edge[:, 1].reshape(-1)].sigmoid().detach().numpy()

        labels = np.r_[np.ones_like(pred_pos_edge), np.zeros_like(pred_neg_edge)]
        preds  = np.r_[pred_pos_edge, pred_neg_edge]

        roc_auc = roc_auc_score(labels,preds)
        precision=average_precision_score(labels, preds)

        return roc_auc,precision




In [0]:
#Training Cell 

my_model = Model(first_layer_dim=30,embedding_dim=15,A_tilda=adj_norm_tensor)


for epoch_counter in range(20):
    
    epoch_loss = my_model.train(features_tensor,adj_label_tensor)
    epoch_roc_auc,epoch_precision = my_model.evaluate(val_edges,val_edges_false)
    
    print("Epoch:", '%04d' % (epoch_counter + 1), "train_loss=", "{:.5f}".format(epoch_loss), "val_roc_auc=", "{:.5f}".format(epoch_roc_auc),
          "val_precision=", "{:.5f}".format(epoch_precision))


print('\n',"#########################################################",'\n')

test_roc_auc,test_precision = my_model.evaluate(test_edges,test_edges_false)
print("test_roc_auc=","{:.5f}".format(test_roc_auc),"test_precision=","{:.5f}".format(epoch_precision))



